In [1]:
#Library imports

import matplotlib.pyplot as plt
import geopandas as gpd
import shapely.wkt
import numpy as np
import pandas as pd
import csv
import re
from shapely.geometry import Point, LineString, Polygon

In [2]:
#This function takes a set of points and a set of non-overlapping polygons and determined
# which polygon each point falls within
def pt_poly_interactions(input_point_df, input_polygon_df, 
                         point_key_col, polygon_key_col, 
                         point_geometry_col, polygon_geometry_col):

    # Convert point data to geopandas dataframe
    pts_working_cols = point_key_col + [point_geometry_col]

    pointsdf = input_point_df[pts_working_cols]
#    pointsdf[point_geometry_col] = gpd.GeoSeries.from_wkt(pointsdf[point_geometry_col])
    gdf_pts = gpd.GeoDataFrame(pointsdf, geometry=point_geometry_col)

    # Add CRS (start with WGS84 to match lat/lon values)
    gdf_pts.set_crs(epsg=4326, inplace=True)

    #Convert polygon data to geopandas dataframe
    polys_working_cols = [polygon_key_col] + [polygon_geometry_col]

    polysdf = input_polygon_df[polys_working_cols]
    polysdf[polygon_geometry_col] = gpd.GeoSeries.from_wkt(polysdf[polygon_geometry_col])
    gdf_polys = gpd.GeoDataFrame(polysdf, geometry=polygon_geometry_col)

    # Add CRS (start with WGS84 to match lat/lon values)
    gdf_polys.set_crs(epsg=4326, inplace=True)

    #Prepare projection (North America Lambert Conformal Conic)
    # This projection is equidistant for measuring between points.
    # Units are in meters
    projout = '+proj=lcc +lat_1=20 +lat_2=60 +lat_0=40 +lon_0=-96 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m no_defs'

    # Convert to Lambert projection
    gdf_pts = gdf_pts.to_crs(projout)

    # Convert to Lambert projection
    gdf_polys = gdf_polys.to_crs(projout)

    #gdf_pts.crs

    #gdf_polys.crs

    gdf_polys = pd.concat([gdf_polys, gdf_polys.centroid], axis=1)
    gdf_polys.columns = polys_working_cols + ['Centroid']
    
    #gdf_polys

    # # Make polygon assignments and calculate distance from centroid

    # Add column to hold the polygon name
    gdf_pts["within_poly"] = ""
    gdf_pts["distance_from_centroid"] = 0.0

    for poly_index,poly_row in gdf_polys.iterrows():
        for pt_index,pt_row in gdf_pts.iterrows():
            if pt_row[2] == "" and pt_row[1].within(poly_row[1]):
                gdf_pts.loc[pt_index, 'within_poly'] = poly_row[0]
                new_point = pt_row[point_geometry_col]
                distance = poly_row['Centroid'].distance(new_point)
                # Convert distance from meters to miles
                distance /= 1609.344
                gdf_pts.loc[pt_index, 'distance_from_centroid'] = distance

    # # Convert back to WGS84 projection

    gdf_pts = gdf_pts.to_crs(epsg=4326)

    return gdf_pts # Compute a Pandas dataframe to return
 

In [3]:
#This is a utility function for converting distance values using different units
def conv_dist(distance_value, units_value):
    # Determine the conversion factor for the specified units (meters are required for this projection)
    if units_value == "mi":
        unit_factor = 1609.344
    elif units_value == "km":
        unit_factor = 1000.0
    elif units_value == "ft":
        unit_factor = 0.3048
    elif units_value == "nm":
        unit_factor = 1852
    elif units_value == "m":
        unit_factor = 1
    else:  # Bad units
        unit_factor = 0

    return distance_value * unit_factor

In [4]:
#This function creates geospatial circle(s) based on center, radius and unit values in the dataset
def gen_geocircle(input_df, key_col, center_col, radius_col, units_col):

    # Convert point data to geopandas dataframe
#    pointsdf = input_df[[key_col, center_col, radius_col, units_col]]

    working_cols = [key_col] + [center_col] + [radius_col] + [units_col]
    return_cols = [key_col] + ["buffer"]
    
    pointsdf = input_df[working_cols]
    
#    pointsdf[center_col] = gpd.GeoSeries.from_wkt(pointsdf[center_col])
    gdf_pts = gpd.GeoDataFrame(pointsdf, geometry=center_col)

    # Add CRS (start with WGS84 to match lat/lon values)
    gdf_pts.set_crs(epsg=4326, inplace=True)

    #Prepare projection (North America Lambert Conformal Conic)
    # This projection is equidistant for measuring between points.
    # Units are in meters
    projout = '+proj=lcc +lat_1=20 +lat_2=60 +lat_0=40 +lon_0=-96 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m no_defs'

    # Convert to Lambert projection
    gdf_pts = gdf_pts.to_crs(projout)

    # -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
    gdf_pts[units_col] = gdf_pts[units_col].str.lower()
    gdf_pts["dist"] = 0

    # -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
    for pt_index,pt_row in gdf_pts.iterrows():
        dvalue = conv_dist(pt_row[2], pt_row[3])

        gdf_pts.loc[pt_index,'dist'] = dvalue

#    gdf2 = pd.merge(gdf_pts[["locid","ArptLocation"]], params[["locid","dist"]], on='locid')

    gdf_pts["buffer"] = gdf_pts[center_col].buffer(gdf_pts['dist'])

    # -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
    gdf_circle = gdf_pts[return_cols]

    gdf_circle = gpd.GeoDataFrame(gdf_circle, geometry='buffer')

    gdf_circle = gdf_circle.to_crs(epsg=4326)

    
    return gdf_circle


In [5]:
#This function determines takes two sets of geospatial objects and determines which ones interact
def find_interactions(geom_set1_df, geom_set2_df,
                      set1_key_cols, set2_key_cols,
                      set1_geometry_col, set2_geometry_col
                      ):

    set1_working_cols = set1_key_cols + [set1_geometry_col]
    set2_working_cols = set2_key_cols + [set2_geometry_col]
    
    # Convert point data to geopandas dataframe

    gdf1 = geom_set1_df[set1_working_cols]
#    gdf1[set1_geometry_col] = gpd.GeoSeries.from_wkt(gdf1[set1_geometry_col])
    gdf1 = gpd.GeoDataFrame(gdf1, geometry=set1_geometry_col)

    # Add CRS (start with WGS84 to match lat/lon values)
    gdf1.set_crs(epsg=4326, inplace=True)
    gdf1_type = gdf1.loc[0, set1_geometry_col].geom_type

    #Convert polygon data to geopandas dataframe
    gdf2 = geom_set2_df[set2_working_cols]
#    gdf2[set2_geometry_col] = gpd.GeoSeries.from_wkt(gdf2[set2_geometry_col])
    gdf2 = gpd.GeoDataFrame(gdf2, geometry=set2_geometry_col)

    # Add CRS (start with WGS84 to match lat/lon values)
    gdf2.set_crs(epsg=4326, inplace=True)

    gdf2_type = gdf2.loc[0, set2_geometry_col].geom_type

    #If datasets are mixed (one polygon and one linestring), ensure polygons are gdf1
    if (gdf1_type == 'LineString' or gdf1_type == 'Point') and gdf2_type == 'Polygon':
        gdf_temp = gdf1
        gdf1 = gdf2
        gdf2 = gdf_temp
        gdf2_type = gdf1_type
        gdf1_type = 'Polygon'
        keys_temp = set1_key_cols
        set1_key_cols = set2_key_cols
        set2_key_cols = keys_temp
        geom_temp = set1_geometry_col
        set1_geometry_col = set2_geometry_col
        set2_geometry_col = geom_temp
        
    # Convert to new equidistant projection

    #Prepare projection (North America Lambert Conformal Conic)
    # This projection is equidistant for measuring between points.
    # Units are in meters
    projout = '+proj=lcc +lat_1=20 +lat_2=60 +lat_0=40 +lon_0=-96 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m no_defs'

    # Convert to Lambert projection
    gdf1 = gdf1.to_crs(projout)

    # Convert to Lambert projection
    gdf2 = gdf2.to_crs(projout)

#    if gdf2_type == 'Point':
#        results = pd.DataFrame(columns=(set1_key_cols + ['Within_polygon']))
#    else:
    results = pd.DataFrame(columns=(set1_key_cols + set2_key_cols + ['Interaction']))
    
    df_index = 0
    
# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
    for gdf1_index,gdf1_row in gdf1.iterrows():
        for gdf2_index,gdf2_row in gdf2.iterrows():
            if gdf1_type == 'Polygon' and gdf2_type == 'LineString':
                interaction = gdf1.loc[gdf1_index, set1_geometry_col].intersects(gdf2.loc[gdf2_index, set2_geometry_col])
                if interaction:
                    start_pt, end_pt = gdf2.loc[gdf2_index, set2_geometry_col].boundary
                    if gdf1.loc[gdf1_index, set1_geometry_col].contains(gdf2.loc[gdf2_index, set2_geometry_col]):
                        interact_str = ['Contains']
                    elif gdf1.loc[gdf1_index, set1_geometry_col].contains(start_pt):
                        interact_str = ['Start']
                    elif gdf1.loc[gdf1_index, set1_geometry_col].contains(end_pt):
                        interact_str = ['End']
                    else:
                        interact_str = ['Intersect']

                    results.loc[df_index] = np.concatenate((gdf1.loc[gdf1_index, set1_key_cols].values,
                                            gdf2.loc[gdf2_index, set2_key_cols].values,
                                            interact_str),axis=None)
                    df_index += 1
            elif gdf1_type == 'LineString' and gdf2_type == 'LineString':
                interaction = gdf1.loc[gdf1_index, set1_geometry_col].intersects(gdf2.loc[gdf2_index, set2_geometry_col])
                if interaction:
                    interact_str = ['Intersect']

                    results.loc[df_index] = np.concatenate((gdf1.loc[gdf1_index, set1_key_cols].values,
                                            gdf2.loc[gdf2_index, set2_key_cols].values,
                                            interact_str),axis=None)
                    df_index += 1
            elif gdf1_type == 'Polygon' and gdf2_type == 'Polygon':
                interaction = gdf1.loc[gdf1_index, set1_geometry_col].intersects(gdf2.loc[gdf2_index, set2_geometry_col])
                if interaction:
                    if gdf1.loc[gdf1_index, set1_geometry_col].contains(gdf2.loc[gdf2_index, set2_geometry_col]):
                        interact_str = ['Contains']
                    else:
                        interact_str = ['Intersect']

                    results.loc[df_index] = np.concatenate((gdf1.loc[gdf1_index, set1_key_cols].values,
                                            gdf2.loc[gdf2_index, set2_key_cols].values,
                                            interact_str),axis=None)
                    df_index += 1
            elif gdf1_type == 'Polygon' and gdf2_type == 'Point':            
                if gdf2.loc[gdf2_index, set2_geometry_col].within(gdf1.loc[gdf1_index, set1_geometry_col]):
                    results.loc[df_index] = np.concatenate((gdf1.loc[gdf1_index, set1_key_cols].values,
                                                            gdf2.loc[gdf2_index, set2_key_cols].values,
                                                            "Contained"),axis=None)
                    df_index += 1

            else:
                print('no match')
                
                
    return results


In [6]:
#Load the datasets
notam_df = pd.read_csv ('data/notam_poly2.csv')

spaceport_df = pd.read_csv ('data/spaceports.csv')

In [7]:
#Convert the latitude/longtitude values to geospatial points
spaceport_df['FacilityLocation'] = [Point(xy) for xy in zip(spaceport_df['LONGITUDE'], spaceport_df['LATITUDE'])]


spaceport_df['radius'] = 5
spaceport_df['units'] = 'mi'

spaceport_df = spaceport_df[spaceport_df['SPACEPORT_REC_ID'] > 1]

sp_df_2 = gen_geocircle(spaceport_df, 'SPACEPORT_REC_ID', 'FacilityLocation', 'radius', 'units')

spaceport_df = pd.merge(spaceport_df, sp_df_2, on='SPACEPORT_REC_ID')

C:\Users\saebasan\anaconda3\envs\spatial\lib\site-packages\pandas\core\dtypes\cast.py:112: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  values = construct_1d_object_array_from_listlike(values)


In [8]:
spaceport_df

,SPACEPORT_REC_ID,SPACEPORT_NAME,LATITUDE,LONGITUDE,OWNER_NAME,FAA_LOC_ID,LOCATION_1,LOCATION_2,FacilityLocation,radius,units,buffer
0,2,Cape Canaveral Air Force Station,28.488889,-80.577778,Department of Defense,XMR,Cape Canaveral,Florida,POINT (-80.57777799999998 28.488889),5,mi,"POLYGON ((-80.49361 28.47556, -80.49550 28.468..."
1,3,Cecil Spaceport,30.218611,-81.876667,Jacksonville Aviation Authority,VQQ,Jacksonville,Florida,POINT (-81.876667 30.218611),5,mi,"POLYGON ((-81.79035 30.20632, -81.79215 30.199..."
2,4,Colorado Air and Space Port,39.785278,-104.543056,Front Range Airport Authority,CFO,Adams County,Colorado,POINT (-104.543056 39.785278),5,mi,"POLYGON ((-104.44354 39.79278, -104.44307 39.7..."
3,5,Corn Ranch,31.423333,-104.758889,Blue Origin,NaN,Van Horn,Texas,POINT (-104.758889 31.423333),5,mi,"POLYGON ((-104.67044 31.43094, -104.67000 31.4..."
4,6,Edwards Air Force Base,34.905556,-117.883611,Department of Defense,EDW,Lancaster,California,POINT (-117.883611 34.905556),5,mi,"POLYGON ((-117.79329 34.92459, -117.79147 34.9..."
5,7,Ellington Airport,29.607222,-95.158889,City of Houston,EFD,Houston,Texas,POINT (-95.158889 29.607222),5,mi,"POLYGON ((-95.07212 29.60645, -95.07263 29.599..."
6,8,John F. Kennedy Space Center,28.524167,-80.650833,NASA,NaN,Merritt Island,Florida,POINT (-80.65083300000001 28.524167),5,mi,"POLYGON ((-80.56661 28.51090, -80.56849 28.503..."
7,9,Mid-Atlantic Regional Spaceport,37.843333,-75.478056,NASA,NaN,Wallops Island,Virginia,POINT (-75.47805599999998 37.843333),5,mi,"POLYGON ((-75.38358 37.82532, -75.38627 37.818..."
8,10,Midland International Air and Space Port,31.942500,-102.201944,City of Midland,MAF,Midland,Texas,POINT (-102.201944 31.9425),5,mi,"POLYGON ((-102.11265 31.94788, -102.11247 31.9..."
9,11,Mojave Air and Space Port,35.059444,-118.151667,East Kern Airport District,MHV,Mojave,California,POINT (-118.151667 35.059444),5,mi,"POLYGON ((-118.06123 35.07871, -118.05937 35.0..."


In [9]:
notam_df['boundary'] = gpd.GeoSeries.from_wkt(notam_df['boundary'])
notam_poly_df = gpd.GeoDataFrame(notam_df, geometry='boundary')

notam_df['valid_geom'] = notam_poly_df['boundary'].is_valid

notam_df

,POLYGON_ID,boundary,valid_geom
0,1,"POLYGON ((-90.18333 44.16667, -89.98333 44.166...",False
1,2,"POLYGON ((-89.00000 44.40000, -89.00000 43.600...",True
2,3,"POLYGON ((-90.58333 44.46667, -90.73889 44.144...",True
3,4,"POLYGON ((-91.08333 44.73333, -90.98333 44.800...",True
4,5,"POLYGON ((-89.98333 44.45000, -89.98333 44.166...",False
...,...,...,...
679105,800654,"POLYGON ((-87.76088 41.79172, -87.76126 41.791...",True
679106,800655,"POLYGON ((-87.75952 41.77930, -87.75912 41.779...",True
679107,800656,"POLYGON ((-87.76088 41.79172, -87.76126 41.791...",True
679108,800658,"POLYGON ((-74.01303 40.19554, -74.00925 40.195...",True


In [10]:
good_geom_df = notam_df[notam_df['valid_geom']]

good_geom_df

,POLYGON_ID,boundary,valid_geom
1,2,"POLYGON ((-89.00000 44.40000, -89.00000 43.600...",True
2,3,"POLYGON ((-90.58333 44.46667, -90.73889 44.144...",True
3,4,"POLYGON ((-91.08333 44.73333, -90.98333 44.800...",True
5,6,"POLYGON ((-60.80000 -31.63315, -60.78305 -31.6...",True
6,7,"POLYGON ((-116.97232 32.82951, -116.97265 32.8...",True
...,...,...,...
679105,800654,"POLYGON ((-87.76088 41.79172, -87.76126 41.791...",True
679106,800655,"POLYGON ((-87.75952 41.77930, -87.75912 41.779...",True
679107,800656,"POLYGON ((-87.76088 41.79172, -87.76126 41.791...",True
679108,800658,"POLYGON ((-74.01303 40.19554, -74.00925 40.195...",True


In [11]:
good_geom_df = good_geom_df.reset_index(drop=True)
spaceport_df = spaceport_df.reset_index(drop=True)

In [12]:
interactions_df = find_interactions(spaceport_df, good_geom_df,
                      ['SPACEPORT_REC_ID'], ['POLYGON_ID'],
                      'buffer', 'boundary'
                      )

interactions_df

,SPACEPORT_REC_ID,POLYGON_ID,Interaction
0,2,680,Intersect
1,2,681,Intersect
2,2,683,Intersect
3,2,703,Intersect
4,2,704,Intersect
...,...,...,...
20693,24,723434,Intersect
20694,24,729953,Intersect
20695,24,730136,Intersect
20696,24,768359,Intersect


In [ ]:
interactions_df.to_csv('interactions.csv', index=False)